In [2]:
import pandas
import sqlite3
import numpy as np

In [3]:
connection = sqlite3.connect('nearby_stars.db')
c = connection.cursor()

## Simbad TAP query information

The <code>simbad_FGK_nobinaries.csv</code> file was created using the ADQL query in the next cell through the Simbad TAP (Table Access Protocol") service. I used TOPCAT to query Simbad and then saved the resulting table as a .csv file, but the TAP service can also be accessed here: https://simbad.u-strasbg.fr/simbad/sim-tap

Information on available Simbad tables is here: http://simbad.u-strasbg.fr/simbad/tap/tapsearch.html

**Simbad TAP query**
<code>
SELECT sub.main_id, sub.ra, sub.dec,  sub_hd.id as hd, sub_gj.id as gj, sub_wds.id as wds, sub.sp_type, sub.sp_bibcode, 
sub.plx_value as plx_value, sub.plx_err, sub.plx_bibcode, 
sub_v.flux as V, sub_v.flux_err as V_err, sub_v.bibcode as V_bibcode, sub_g.flux as G, sub_g.flux_err as G_err, sub_g.bibcode as G_bibcode
FROM (
SELECT basic.main_id, basic.oid, basic.ra, basic.dec, (alltypes.otypes || '|' || otypedef.otype_shortname) AS otypes_all, basic.sp_type, basic.sp_bibcode, 
basic.plx_value, basic.plx_err, basic.plx_bibcode, allfluxes.V, allfluxes.G
FROM alltypes
    INNER JOIN basic ON basic.oid = alltypes.oidref
    LEFT OUTER JOIN otypedef ON basic.otype = otypedef.otype
    LEFT OUTER JOIN h_link ON h_link.parent = alltypes.oidref
    LEFT OUTER JOIN basic AS basic_child ON h_link.child = basic_child.oid
    LEFT OUTER JOIN alltypes AS alltypes_child ON h_link.child = alltypes_child.oidref
    LEFT OUTER JOIN allfluxes ON allfluxes.oidref = basic.oid
WHERE basic.plx_value > 0
AND 1/(basic.plx_value\*1e-3) &#60; 20
AND (basic.sp_type LIKE 'F%' OR basic.sp_type LIKE 'G%' OR basic.sp_type LIKE 'K%')
AND ((basic.sp_type NOT LIKE '%II%') 
    OR (basic.sp_type NOT LIKE '%I%' AND basic.sp_type NOT LIKE '%V%'))
AND ((alltypes.otypes || '|' || otypedef.otype_shortname LIKE '%\*\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%SB%' 
    AND alltypes_child.otypes LIKE '%Pl%') 
    OR (alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%\*\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%SB\*%' 
    AND alltypes.otypes || '|' || otypedef.otype_shortname NOT LIKE '%EB\*%'))
AND (basic.main_id NOT LIKE '% A' AND basic.main_id NOT LIKE '% B' AND basic.main_id NOT LIKE '% C' AND basic.main_id NOT LIKE '% D')
GROUP BY basic.main_id,basic.oid, basic.ra, basic.dec, otypes_all, basic.sp_type, basic.plx_value, basic.plx_err, allfluxes.V, allfluxes.G
) AS sub
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'WDS%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_wds ON sub.oid = sub_wds.oid
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'HD%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_hd ON sub.oid = sub_hd.oid
LEFT OUTER JOIN (
    SELECT basic.oid, STRING_AGG(ident.id,',') as id
    FROM basic
        INNER JOIN ident ON basic.oid = ident.oidref
    WHERE (ident.id LIKE 'GJ%')
    AND ident.id NOT LIKE '%.0'
    GROUP BY basic.oid
    ) AS sub_gj ON sub.oid = sub_gj.oid
LEFT OUTER JOIN (
    SELECT basic.oid, flux.filter, flux.flux, flux.flux_err, flux.bibcode
    FROM basic INNER JOIN flux ON flux.oidref = basic.oid 
    WHERE flux.filter = 'V') AS sub_v ON sub.oid = sub_v.oid
LEFT OUTER JOIN (
    SELECT basic.oid, flux.filter, flux.flux, flux.flux_err, flux.bibcode
    FROM basic INNER JOIN flux ON flux.oidref = basic.oid 
    WHERE flux.filter = 'G') AS sub_g ON sub.oid = sub_g.oid
GROUP BY sub.main_id, sub.ra, sub.dec,  hd, gj, wds, sub.sp_type, sub.sp_bibcode, 
plx_value, sub.plx_err, sub.plx_bibcode,
 V, V_err, V_bibcode, G, G_err, G_bibcode
HAVING ((sub_hd.id NOT LIKE '%A%' AND sub_hd.id NOT LIKE '%B%') OR sub_hd.id IS NULL)
    AND ((sub_gj.id NOT LIKE '%A%' AND sub_gj.id NOT LIKE '%B%') OR sub_gj.id IS NULL)
ORDER BY plx_value DESC
</code>

## Add TAP query saved in .csv file to database

In [3]:
dat = pandas.read_csv('simbad_FGK_nobinaries.csv')

In [5]:
#c.execute('DROP TABLE IF EXISTS stars_FGK_nobinaries')

In [6]:
dat.to_sql('stars_FGK_nobinaries', connection, index_label='sid')

In [11]:
connection.commit()

## Make sid ("star id") column a Primary Key

Create a temporary backup table.

In [4]:
cmd = '''
CREATE TEMPORARY TABLE IF NOT EXISTS stars_backup(sid, main_id, ra, dec, hd, gj, wds, sp_type, 
sp_bibcode, plx_value, plx_err, plx_bibcode, v, v_err,
       v_bibcode, g, g_err, g_bibcode);
'''

c.execute(cmd)

In [5]:
cmd = '''
INSERT INTO stars_backup SELECT * FROM stars_FGK_nobinaries;
'''

c.execute(cmd)

In [6]:
cmd = '''
DROP TABLE stars_FGK_nobinaries
'''
c.execute(cmd)

In [7]:
cmd = '''
CREATE TABLE stars_FGK_nobinaries (
sid INTEGER PRIMARY KEY AUTOINCREMENT,
main_id TEXT,
ra TEXT,
dec TEXT,
hd TEXT,
gj TEXT,
wds TEXT,
sp_type TEXT,
sp_bibcode TEXT,
plx_value REAL,
plx_err REAL,
plx_bibcode TEXT,
v REAL,
v_err REAL,
v_bibcode TEXT,
G REAL,
G_err REAL,
G_bibcode TEXT
)
'''
c.execute(cmd)

In [8]:
cmd = '''
INSERT INTO stars_FGK_nobinaries SELECT * FROM stars_backup
'''
c.execute(cmd)

In [10]:
connection.commit()

In [13]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
c.fetchall()

[(u'sqlite_sequence',),
 (u'stars_simbad',),
 (u'ages',),
 (u'stars_FGK_nobinaries',)]

## Look at the new table

In [11]:
c.execute('PRAGMA table_info(stars_FGK_nobinaries);')
c.fetchall()

[(0, u'sid', u'INTEGER', 0, None, 1),
 (1, u'main_id', u'TEXT', 0, None, 0),
 (2, u'ra', u'TEXT', 0, None, 0),
 (3, u'dec', u'TEXT', 0, None, 0),
 (4, u'hd', u'TEXT', 0, None, 0),
 (5, u'gj', u'TEXT', 0, None, 0),
 (6, u'wds', u'TEXT', 0, None, 0),
 (7, u'sp_type', u'TEXT', 0, None, 0),
 (8, u'sp_bibcode', u'TEXT', 0, None, 0),
 (9, u'plx_value', u'REAL', 0, None, 0),
 (10, u'plx_err', u'REAL', 0, None, 0),
 (11, u'plx_bibcode', u'TEXT', 0, None, 0),
 (12, u'v', u'REAL', 0, None, 0),
 (13, u'v_err', u'REAL', 0, None, 0),
 (14, u'v_bibcode', u'TEXT', 0, None, 0),
 (15, u'G', u'REAL', 0, None, 0),
 (16, u'G_err', u'REAL', 0, None, 0),
 (17, u'G_bibcode', u'TEXT', 0, None, 0)]

In [17]:
c.execute('SELECT * FROM stars_FGK_nobinaries')
rows = c.fetchall()
print len(rows)
print
for row in rows[0:10]:
    print row

227

(0, u'* eps Eri', 53.2326873475, -9.458258656944443, u'HD  22049', u'GJ   144', u'WDS J03329-0927A,WDS J03329-0927Aa,Ab', u'K2V', u'1989ApJS...71..245K', 310.94, 0.16, u'2007A&A...474..653V', 3.73, None, u'2002yCat.2237....0D', 3.3691, 0.004, u'2018yCat.1345....0G')
(1, u'* tau Cet', 26.01701426083333, -15.937479597777779, u'HD  10700', u'GJ    71', u'WDS J01441-1556A', u'G8V', u'1989ApJS...71..245K', 273.96, 0.17, u'2007A&A...474..653V', 3.5, None, u'2002yCat.2237....0D', 3.1336, 0.0075, u'2018yCat.1345....0G')
(2, u'* e Eri', 49.98187889666666, -43.06978263750001, u'HD  20794', u'GJ   139', None, u'G6V', u'1989ApJS...71..245K', 165.47, 0.19, u'2007A&A...474..653V', 4.27, None, u'2002yCat.2237....0D', 3.9814, 0.0037, u'2018yCat.1345....0G')
(3, u'* del Pav', 302.18170612375, -66.1820675763889, u'HD 190248', u'GJ   780', None, u'G8IV', u'2006AJ....132..161G', 163.71, 0.17, u'2007A&A...474..653V', 3.56, None, u'2002yCat.2237....0D', 3.2664, 0.0025, u'2018yCat.1345....0G')
(4, u'HD 

## Close connection when done

In [14]:
connection.close()